In [7]:
import torch
from torch import nn
class MyLinear(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        weight = torch.randn(d_out, d_in)
        self.weight = nn.Parameter(weight)

net = MyLinear(2,2)
net.to(device="cuda:0")
net.weight.device

device(type='cuda', index=0)

In [14]:
print(" !".encode("utf-8"))

b' !'


In [16]:
test_string = "hello!     !"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)

b'hello!     !'
